In [37]:
class llg2reg():
    def __init__(self, grammer):
        self.grammer = grammer
        self.prod_rules = list(map(lambda x : x.strip(), grammer.split(",")))
        self.trans = {}
        self.rev_trans = {}
        self.graph = {}
        self.rev_graph = {}
        self.intial_state = ''
        self.states = set()
        self.reg = ""

    def create_data(self):
        self.trans = {}
        self.rev_trans = {}
        self.graph = {}
        self.rev_graph = {}
        self.intial_state = ''
        self.states = set()

        for i, rule in enumerate(self.prod_rules):
            start_symbol, exps = list(map(lambda x : x.strip(" "), rule.split("->")))
            if i == 0: self.intial_state = start_symbol

            if "/" in exps: exps = list(map(lambda x : x.strip(" "), exps.split("/")))
            else: exps = [exps]
            
            for exp in exps:
                n_terminal = list(filter(lambda x : x.isupper(), exp))
                ind = len(exp)
                
                for i, c in enumerate(exp):
                    if c.isupper() : ind = i; break
                    
                if len(n_terminal) == 0: n_terminal.append("ff")

                if ind != -1: terminal = [exp[:ind]]
                else: terminal = []

                for t in n_terminal:
                    self.trans[start_symbol] = self.trans.get(start_symbol, [])
                    self.trans[start_symbol].append({"transition": terminal, "node": t})

        self.trans["ss"] = [{'transition': ['e'], 'node': self.intial_state}]

        for state in self.trans:
            for eq in self.trans[state]:
                self.rev_trans[eq["node"]] = self.rev_trans.get(eq["node"], [])
                self.rev_trans[eq["node"]].append({"transition": eq["transition"], "node": state})

        for key in self.trans:
            self.states.add(key)
        for key in self.rev_trans:
            self.states.add(key)


        self.graph = {key:{key:[] for key in self.states} for key in self.states}

        for nd in self.trans:
            for tn in self.trans[nd]:
                self.graph[nd][tn['node']].extend(tn["transition"])



    def log_data(self, full=False):
        print("------- Graph -------")
        for nd in self.graph:
            for tn in self.graph[nd]:
                if len(self.graph[nd][tn]) != 0 or full: 
                    print(f"{nd} -> {tn} ==" + str(self.graph[nd][tn]))
        print("\n\n")
        print("------- Reverse Graph -------")
        for nd in self.rev_graph:
            for tn in self.rev_graph[nd]:
                if len(self.rev_graph[nd][tn]) != 0 or full: 
                    print(f"{nd} -> {tn} ==" + str(self.rev_graph[nd][tn]))

    def create_reg(self):
        self.create_data()
        print(self.states)
        for nd in self.states:
            if nd in ["ss", "ff"]: continue
            
            if len(self.graph[nd][nd]) == 0: re_m = ""
            else: re_m = "(" + "+".join(self.graph[nd][nd]) + ")*"
            
            self.graph[nd][nd] = []
            
            nexts = set([x for x in self.graph[nd] if len(self.graph[nd][x]) > 0])
            prevs = set()

            for i in self.graph:
                for j in self.graph[i]:
                    if len(self.graph[i][j])>0 and j==nd: prevs.add(i)
            
            for pr in prevs:
                for nx in nexts:
                    if pr == nd or nx == nd: continue
                    print(pr, nx, nd)
                    if len(self.graph[pr][nd]) == 0:re_p = ""
                    else: re_p = "(" + "+".join(self.graph[pr][nd]) + ")"
                    if len(self.graph[nd][nx]) == 0: re_n = ""
                    else: re_n = "(" + "+".join(self.graph[nd][nx]) + ")"
                    
                    self.graph[nd][nx] = []
                    
                    if len(re_p + re_m + re_n) != 0:
                        self.graph[pr][nx].append("(" + re_p + re_m + re_n + ")")

                self.graph[pr][nd] = []
            print(self.graph)

        self.reg = self.graph["ss"]["ff"]

        return self.reg

    

# S -> 00B/11S, B -> 0B/1B/0/1

In [38]:
conv = llg2reg("S -> 00B/11S, B -> 0B/1B/0/1") 
conv.create_data()
conv.log_data()
conv.create_reg()

------- Graph -------
S -> S ==['11']
S -> B ==['00']
ss -> S ==['e']
B -> ff ==['0', '1']
B -> B ==['0', '1']



------- Reverse Graph -------
S -> S ==['11']
S -> ss ==['e']
ff -> B ==['0', '1']
B -> S ==['00']
B -> B ==['0', '1']
{'S', 'ff', 'ss', 'B'}
ss B S
{'S': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ff': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ss': {'S': [], 'ff': [], 'ss': [], 'B': ['((e)(11)*(00))']}, 'B': {'S': [], 'ff': ['0', '1'], 'ss': [], 'B': ['0', '1']}}
ss ff B
{'S': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ff': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ss': {'S': [], 'ff': ['((((e)(11)*(00)))(0+1)*(0+1))'], 'ss': [], 'B': []}, 'B': {'S': [], 'ff': [], 'ss': [], 'B': []}}


['((((e)(11)*(00)))(0+1)*(0+1))']

In [25]:
reg_grm = input()
rules = list(map(lambda x : x.strip(), reg_grm.split(",")))
eqs = {}
intial_state = ''

for i, rule in enumerate(rules):
    start_symbol, poss = map(lambda x : x.strip(), rule.split("->"))
    if i == 0: intial_state = start_symbol

    if "/" in poss: exps = map(lambda x : x.strip(), poss.split("/"))
    else: exps = [poss]
        
    for exp in exps:
        n_terminal = list(filter(lambda x : x.isupper(), exp))
        ind = len(eqs)
        for i, c in enumerate(exp):
            if c.isupper() : ind = i; break
        if len(n_terminal) == 0: n_terminal.append("ff")
        if ind != -1:
            terminal = [exp[:ind]]
        else: terminal = []

        for t in n_terminal:
            eqs[start_symbol] = eqs.get(start_symbol, [])
            eqs[start_symbol].append({"transition": terminal, "node": t})

eqs["ss"] = [{'transition': ['e'], 'node': intial_state}]

rev_eqs = {}

for state in eqs:
    for eq in eqs[state]:
        rev_eqs[eq["node"]] = rev_eqs.get(eq["node"], [])
        rev_eqs[eq["node"]].append({"transition": eq["transition"], "node": state})

states = set()
for key in eqs:
    states.add(key)
for key in rev_eqs:
    states.add(key)

grp = {key:{key:[] for key in states} for key in states}
rev_grp = {key:{key:[] for key in states} for key in states}

for nd in eqs:
    for tn in eqs[nd]:
        grp[nd][tn['node']].extend(tn["transition"])

for nd in rev_eqs:
    for tn in rev_eqs[nd]:
        rev_grp[nd][tn['node']].extend(tn["transition"])

print(grp)
print(eqs)
print(rev_eqs)
print(states)
# S -> 00B/11S, B -> 0B/1B/0/1

{'S': {'S': ['11'], 'ff': [], 'ss': [], 'B': ['00']}, 'ff': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ss': {'S': ['e'], 'ff': [], 'ss': [], 'B': []}, 'B': {'S': [], 'ff': ['0', '1'], 'ss': [], 'B': ['0', '1']}}
{'S': [{'transition': ['00'], 'node': 'B'}, {'transition': ['11'], 'node': 'S'}], 'B': [{'transition': ['0'], 'node': 'B'}, {'transition': ['1'], 'node': 'B'}, {'transition': ['0'], 'node': 'ff'}, {'transition': ['1'], 'node': 'ff'}], 'ss': [{'transition': ['e'], 'node': 'S'}]}
{'B': [{'transition': ['00'], 'node': 'S'}, {'transition': ['0'], 'node': 'B'}, {'transition': ['1'], 'node': 'B'}], 'S': [{'transition': ['11'], 'node': 'S'}, {'transition': ['e'], 'node': 'ss'}], 'ff': [{'transition': ['0'], 'node': 'B'}, {'transition': ['1'], 'node': 'B'}]}
{'S', 'ff', 'ss', 'B'}


In [26]:
print(grp)
for nd in states:
    if nd in ["ss", "ff"]: continue
    
    if len(grp[nd][nd]) == 0: re_m = ""
    else: re_m = "(" + "+".join(grp[nd][nd]) + ")*"
    grp[nd][nd] = []
    
    nexts = set([x for x in grp[nd] if len(grp[nd][x]) > 0])
    prevs = set([x["node"] for x in rev_eqs[nd]])
        
    print(nexts)
    print(prevs) 
    
    for pr in prevs:
        for nx in nexts:
            if pr == nd or nx == nd: continue
            print(pr, nx)
            if len(grp[pr][nd]) == 0:re_p = ""
            else: re_p = "(" + "+".join(grp[pr][nd]) + ")"
            if len(grp[nd][nx]) == 0: re_n = ""
            else: re_n = "(" + "+".join(grp[nd][nx]) + ")"
            
            grp[nd][nx] = []
            
            if len(re_p + re_m + re_n) != 0:
                grp[pr][nx].append("(" + re_p + re_m + re_n + ")")
        grp[pr][nd] = []

    print(grp["ss"]["ff"])

{'S': {'S': ['11'], 'ff': [], 'ss': [], 'B': ['00']}, 'ff': {'S': [], 'ff': [], 'ss': [], 'B': []}, 'ss': {'S': ['e'], 'ff': [], 'ss': [], 'B': []}, 'B': {'S': [], 'ff': ['0', '1'], 'ss': [], 'B': ['0', '1']}}
{'B'}
{'S', 'ss'}
ss B
[]
{'ff'}
{'S', 'B'}
S ff
[]


In [68]:
 B->aA/aB/bB, A->e ((e)(a+b)*(((a)(e)))
 A->a,A->aB,A->e
S -> 00B/11S, B -> 0B/1B/0/1  (((e)(11)*(((00)(0+1)*(0+1))))

SyntaxError: invalid syntax (Temp/ipykernel_13680/3074289860.py, line 1)

In [ ]:
dfa = DFA([A, B], [a, b], B, [A], )